In [63]:
import numpy as np 
import random 
from scipy.special import logit, expit
from scripts import viterby


def expirement(N = 1000, n = 1000, mode = True, trn = None):
    if trn is None:
        trn = [[0.9, 0.1],
                [0.1, 0.9]]
    if mode:
        s = 0
        for _ in range(N):
            d1 = np.array(distribution(6, 1000)/1000)
            d2 = expit(logit(d1) + np.random.normal(scale = 0.01, size = 6))

            symbols = [i+1 for i in range(6)]
            dist1 = dict(zip(symbols,d1))
            dist2 = dict(zip(symbols,d2))
            ems = [dist1,dist2]
            s+= test(trn, ems, n)
        return s/N
    
    s = 0
    cnt_nans = 0
    for i in range(N):
        symbols = [i+1 for i in range(6)]
        a = np.random.randint(1000, 100000)
        d = [1/a, 1/a,1/a, 1/a,1/a, 1-5/a]
        random.shuffle(d)
        ems = [
            {1:1/6,2:1/6,3:1/6,4:1/6,5:1/6,6:1/6},
            dict(zip(symbols,d))]
        metrics = test(trn, ems, n)
        if metrics.isna().sum().sum()>0:
            cnt_nans -=1 
        s+= metrics
    return s/(N + cnt_nans)

def generate(trn, ems, n = 10000):
    states = [np.random.randint(2)]
    for i in range(1,n):
        state = states[-1]
        prob = trn[state][1]
        states.append(np.random.binomial(1, prob, size=None))
    symbols = []
    all_symbols = [i+1 for i in range(len(ems[0]))]
    for i in range(n):
        symbol = np.random.choice(all_symbols,1,  ems[states[i]])
        symbols.append(symbol[0])
    return states, symbols


def test(trn, ems, n):
    states, symbols = generate(trn, ems, n)
    prediction = viterby(
        observations=symbols, 
        init_prob=[1/2, 1/2],
        transition_prob=trn,
        emision_prob=ems
    )
    n_pre = sum(np.array(states))
    n_rec = sum(np.array(prediction)==np.array(states))
    accuracy = (np.array(prediction)==np.array(states)).sum()/n
    precision = ((np.array(prediction)==np.array(states)) & (np.array(states)==1)).sum()/n_pre
    # print(((np.array(prediction)==np.array(states)) & (np.array(states)==1)).sum())
    # print(sum(prediction))
    # print(states)
    recall = precision*n_pre/n_rec
    # print(n_pre, precision, n_rec, recall)

    metrics = pd.DataFrame({
        'accuracy':accuracy,
        'precision':precision,
        'recall':recall}, index = ['metrics'])
    return metrics


def distribution(n, total):
    """Return a randomly chosen list of n positive integers summing to total.
    Each such list is equally likely to occur."""

    dividers = sorted(random.sample(range(1, total), n - 1))
    return np.array([a - b for a, b in zip(dividers + [total], [0] + dividers)])


# Описание экспериментов

Цель экспериментов - проверить точность алгоритма Витерби при разных матрицах эмиссий, но с одинаковыми матрицам переходов. В качестве матрицы переходов зафиксировал:
```
[[0.95, 0.05],
 [0.1,  0.9 ]]
```
Рассмотрел два класса матриц эмиссий:
1. С нулевой дивергенцией, на каждой итерации эксперимента генирировал одинаковое для обоих состояний произвольное распределение из 6 символов.
2. С большой дивергенцией для:
    
 - первого состояния распределение на каждой итерации равно `[1/6,1/6,1/6,1/6,1/6,1/6]`
 - второго состояяния распределения имеет очень большую вероятность в одном из символов и одинаковую в остальных. Например,`[495/500,1/500,1/500,1/500,1/500, 1/500]`

Итерация эксперимента:
1. Генерируются параметры скрытой марковской цепи в соответствии с правилами выше для эмиссий с большой или малой дивергенцией
2. Генерируется последовательность скрытых состояний и символов длиной `100`
3. Применяется алгоритм Витерби и считаем процент верно предсказанных скрытых состояний

В каждом экперименте `100 000` итераций, полсе того, как все они сработали, считаем и выводим среднюю точность алгоритма Витерби по всем итерациями.


## Маленькая дивергенция

In [64]:
trn = [[0.9, 0.1],
       [0.1, 0.9]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.499348,0.4981,0.4981


In [65]:
trn = [[0.8, 0.2],
       [0.2, 0.8]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.498212,0.5,0.5


In [66]:
trn = [[0.7, 0.3],
       [0.3, 0.7]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.498947,0.5029,0.5029


In [70]:
trn = [[0.6, 0.4],
       [0.4, 0.6]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.501058,0.499,0.499


In [71]:
trn = [[0.5, 0.5],
       [0.5, 0.5]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.499857,0.496959,0.497635


In [78]:
trn = [[0.4, 0.6],
       [0.6, 0.4]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.499171,0.499122,0.499739


In [79]:
trn = [[0.3, 0.7],
       [0.7, 0.3]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.499128,0.499076,0.500422


In [80]:
trn = [[0.2, 0.8],
       [0.8, 0.2]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.500263,0.500226,0.500091


In [81]:
trn = [[0.1, 0.9],
       [0.9, 0.1]]
expirement(10000,100, trn = trn)

,accuracy,precision,recall
metrics,0.501981,0.501977,0.499772


# Большая дивергенция

In [67]:
trn = [[0.9, 0.1],
       [0.1, 0.9]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.500752,0.009,0.010208


In [68]:
trn = [[0.8, 0.2],
       [0.2, 0.8]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.500187,0.029017,0.030642


In [69]:
trn = [[0.7, 0.3],
       [0.3, 0.7]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.498936,0.029173,0.030005


In [72]:
trn = [[0.6, 0.4],
       [0.4, 0.6]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.500271,0.029387,0.029675


In [73]:
trn = [[0.5, 0.5],
       [0.5, 0.5]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.499873,0.028845,0.028847


In [74]:
trn = [[0.4, 0.6],
       [0.6, 0.4]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.50042,0.029144,0.028948


In [75]:
trn = [[0.3, 0.7],
       [0.7, 0.3]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.500341,0.0291,0.028836


In [76]:
trn = [[0.2, 0.8],
       [0.8, 0.2]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.499936,0.029233,0.028952


In [77]:
trn = [[0.1, 0.9],
       [0.9, 0.1]]
expirement(10000,100, trn = trn, mode = False)

,accuracy,precision,recall
metrics,0.499949,0.030307,0.0299
